In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
from huggingface_hub import login
login("token")


In [3]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

wb_token = user_secrets.get_secret("wandb")
wandb.login(key=wb_token)
run = wandb.init(
    project="Fine-tune-Emotion-Detector-Llama-2-7b-ft-instruct-es-testing-v5", 
    job_type="training", 
    anonymous="allow"
)

print("Run iniciado correctamente.")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: joseph-rios (joseph-rios-unl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Run iniciado correctamente.


In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Suppress TensorFlow logs
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
import torch
import bitsandbytes as bnb
from datasets import load_dataset, DatasetDict, Dataset
from collections import Counter
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

E0000 00:00:1747683440.444680      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747683440.515286      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Definir la función para hacer un submuestreo seguro
def submuestreo_por_categoria(df, categoria_col, tamaño_muestra=5610):
    # Crear una lista para almacenar las muestras de cada categoría
    muestras = []
    
    # Iterar sobre cada categoría única en el DataFrame
    for categoria in df[categoria_col].unique():
        # Filtrar los datos de esa categoría
        categoria_df = df[df[categoria_col] == categoria]
        
        # Si la categoría tiene más de 'tamaño_muestra' registros, submuestrear
        if len(categoria_df) >= tamaño_muestra:
            muestra_categoria = categoria_df.sample(n=tamaño_muestra, random_state=42)
        else:
            # Si la categoría tiene menos registros que 'tamaño_muestra', tomamos todos los registros
            muestra_categoria = categoria_df
        
        # Agregar la muestra de la categoría a la lista
        muestras.append(muestra_categoria)
    
    # Concatenar todas las muestras y devolver el DataFrame resultante
    return pd.concat(muestras)

In [7]:
# Model from Hugging Face hub
base_model = "clibrain/Llama-2-7b-ft-instruct-es"

# New instruction dataset
emotion_dataset = "Joseph7D/emotion-dataset-v2"

# Fine-tuned model
new_model = "llama-2-7b-emotion-detector-v5"

dataset = load_dataset(emotion_dataset)

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-2eaa7208be503bdb.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

(…)-00000-of-00001-a8843463f30ab62e.parquet:   0%|          | 0.00/234k [00:00<?, ?B/s]

(…)-00000-of-00001-6315e716207ff76f.parquet:   0%|          | 0.00/234k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26934 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3366 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3372 [00:00<?, ? examples/s]

In [8]:
X_train = dataset['train'].to_pandas()        # 80% = 2688 ejemplos
# 
# X_train = submuestreo_por_categoria(X_train, categoria_col='emotion', tamaño_muestra=100)
# X_train = X_train.reset_index(drop=True)
# X_train = X_train.sample(frac=1).reset_index(drop=True)
# 

X_eval  = dataset['validation'].to_pandas()   # 10% = 3366 ejemplos
# X_eval = submuestreo_por_categoria(X_eval, categoria_col='emotion', tamaño_muestra=10)
# X_eval = X_eval.reset_index(drop=True)
# X_eval = X_eval.sample(frac=1).reset_index(drop=True)

X_test  = dataset['test'].to_pandas()         # 10% = 3366 ejemplos
# X_test = submuestreo_por_categoria(X_test, categoria_col='emotion', tamaño_muestra=10)
# X_test = X_test.reset_index(drop=True)
# X_test = X_test.sample(frac=1).reset_index(drop=True)

# Confirmar distribuciones
print("Distribución en X_train:")
print(X_train['emotion'].value_counts())

print("\nDistribución en X_eval:")
print(X_eval['emotion'].value_counts())

print("\nDistribución en X_test:")
print(X_test['emotion'].value_counts())

Distribución en X_train:
emotion
ira         4489
miedo       4489
alegría     4489
neutral     4489
tristeza    4489
disgusto    4489
Name: count, dtype: int64

Distribución en X_eval:
emotion
ira         562
tristeza    562
alegría     562
neutral     562
disgusto    562
miedo       562
Name: count, dtype: int64

Distribución en X_test:
emotion
neutral     561
tristeza    561
ira         561
alegría     561
miedo       561
disgusto    561
Name: count, dtype: int64


In [9]:
def generate_prompt(row):
    return f"<s>[INST] Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. " \
               f"Responde únicamente con la emoción correspondiente.\n\n" \
               f"Texto: \"{row['text']}\" [/INST] {row['emotion']} </s>"

def generate_test_prompt(row):
    return f"<s>[INST] Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. " \
               f"Responde únicamente con la emoción correspondiente.\n\n" \
               f"Texto: \"{row['text']}\" [/INST] </s>"

In [10]:
# Generate prompts for training and evaluation data
X_train.loc[:,'statement'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'statement'] = X_eval.apply(generate_prompt, axis=1)



In [11]:
# X_test  = dataset['test'].to_pandas()         # 10% = 3366 ejemplos
# X_test = submuestreo_por_categoria(X_test, categoria_col='emotion', tamaño_muestra=10)
# X_test = X_test.reset_index(drop=True)
# X_test = X_test.sample(frac=1).reset_index(drop=True)

In [12]:
# Generate test prompts and extract true labels
y_true = X_test.loc[:,'emotion']
# X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])
X_test.loc[:,'statement'] = X_test.apply(generate_test_prompt, axis=1)

In [13]:
# Checking the size of the training dataset
print("Training dataset size:", X_train.shape)

# Checking the size of the evaluation dataset
print("Evaluation dataset size:", X_eval.shape)

# Checking the size of the test dataset
print("Test dataset size:", X_test.shape)

# Checking the size of the true labels
print("True labels size:", y_true.shape)


Training dataset size: (26934, 3)
Evaluation dataset size: (3372, 3)
Test dataset size: (3366, 3)
True labels size: (3366,)


In [14]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["statement"]])
eval_data = Dataset.from_pandas(X_eval[["statement"]])
test_data = Dataset.from_pandas(X_test[["statement"]])
train_data['statement'][168]

'<s>[INST] Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. Responde únicamente con la emoción correspondiente.\n\nTexto: "¡Qué alegría poder compartir este momento contigo!" [/INST] alegría </s>'

## Cargar Modelo

In [15]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [16]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

In [17]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

## Evaluar antes del ajuste

In [18]:
logging.set_verbosity(logging.CRITICAL)

# Suprime mensajes de advertencia
logging.set_verbosity(logging.CRITICAL)

# Genera el resultado usando el pipeline
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# Crear el pipeline para generación de texto
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.95
)


# Diseñar el prompt para clasificación de emociones
prompt = (
    "Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. Responde únicamente con la emoción correspondiente.\n"
    "Texto: No es un corte. ¡Es una herida! ¡Es más grande que un corte! ¡Ay! Necesito un médico\n"
    # "Emoción:"
)
# Generar respuesta con el pipeline
output = pipe(prompt, max_new_tokens=10)
generated_text = output[0]["generated_text"] if output else ""
print("Texto generado:", generated_text)

# Definir las etiquetas válidas
etiquetas_validas = ["ira", "disgusto", "tristeza", "alegría", "miedo", "neutral"]

# Extraer la parte relevante después de "Emoción:"
respuesta_generada = generated_text.split("### Respuesta:")[-1].strip().lower()

# Validar la etiqueta generada
respuesta = next((etiqueta for etiqueta in etiquetas_validas if etiqueta in respuesta_generada), "no clasificado")

print("Etiqueta clasificada:", respuesta)

Texto generado: Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. Responde únicamente con la emoción correspondiente.
Texto: No es un corte. ¡Es una herida! ¡Es más grande que un corte! ¡Ay! Necesito un médico

### Respuesta: Miedo.
Etiqueta clasificada: miedo


In [19]:
eval_data['statement'][5]

'<s>[INST] Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. Responde únicamente con la emoción correspondiente.\n\nTexto: "No tolero la falta de sinceridad honestidad en las relaciones interpersonales" [/INST] ira </s>'

## Entrenar (pendiente)

In [23]:
import bitsandbytes as bnb
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['v_proj', 'o_proj', 'down_proj', 'gate_proj', 'k_proj', 'up_proj', 'q_proj']

In [24]:
def preprocess_function(examples):
    return tokenizer(examples["statement"], truncation=True, padding=True)

train_data = train_data.map(preprocess_function, batched=True)
eval_data = eval_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/26934 [00:00<?, ? examples/s]

Map:   0%|          | 0/3372 [00:00<?, ? examples/s]

In [25]:
# Set training parameters
output_dir="/kaggle/working/Llama-2-7b-ft-instruct-es-fine-tuned-testing5"
# Load LoRA configuration
# peft_args = LoraConfig(
#     lora_alpha=16,
#     lora_dropout=0.1,
#     r=64,
#     bias="none",
#     task_type="CAUSAL_LM",
# )
peft_args = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=16,
    bias="lora_only",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_params = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=2000,             # guardar cada 1000 pasos
    save_total_limit=1,          # mantener solo el checkpoint más reciente
    logging_steps=50,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    report_to="wandb",                  # report metrics to w&b
    lr_scheduler_type="constant",
    disable_tqdm=False,           
    # logging_dir="./logs",
    eval_strategy="steps",
    eval_steps=2000,
    # load_best_model_at_end=True,
    # metric_for_best_model="loss",
    # save_total_limit=3,
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_args,
    args=training_params,
)

Truncating train dataset:   0%|          | 0/26934 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/3372 [00:00<?, ? examples/s]

No se busca una clasificacion explicita, sino mas bien una generaricon condicionada

El warning de label_names:

"No label_names provided for model class PeftModelForCausalLM..."

puede ser ignorado, porque el modelo no está haciendo clasificación explícita, sino generación condicionada.

💡 Trainer espera clasificación si se usa compute_metrics, pero aquí es generación. Así que se puede usar un Trainer personalizado o Seq2SeqTrainer, o incluso un bucle de entrenamiento.

In [26]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


TrainOutput(global_step=3366, training_loss=0.40009446846751745, metrics={'train_runtime': 34614.2861, 'train_samples_per_second': 0.778, 'train_steps_per_second': 0.097, 'total_flos': 1.9798459371405312e+17, 'train_loss': 0.40009446846751745})

In [ ]:
# # Generar predicciones
# y_true = y_true.to_frame()
# y_true.name = 'emotion'

In [27]:
# Directorio donde se guardará el modelo
output_dir = "/kaggle/working/llama-2-7b-emotion-detector-v5"

# Guarda sólo los pesos del modelo y el tokenizer
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
!zip -r /kaggle/working/modelo-ajustado-v5.zip /kaggle/working/llama-2-7b-emotion-detector-v5

  adding: kaggle/working/llama-2-7b-emotion-detector-v5/ (stored 0%)
  adding: kaggle/working/llama-2-7b-emotion-detector-v5/special_tokens_map.json (deflated 73%)
  adding: kaggle/working/llama-2-7b-emotion-detector-v5/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama-2-7b-emotion-detector-v5/adapter_config.json (deflated 55%)
  adding: kaggle/working/llama-2-7b-emotion-detector-v5/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/llama-2-7b-emotion-detector-v5/README.md (deflated 66%)
  adding: kaggle/working/llama-2-7b-emotion-detector-v5/tokenizer_config.json (deflated 68%)


In [28]:
logging.set_verbosity(logging.CRITICAL)

# Suprime mensajes de advertencia
logging.set_verbosity(logging.CRITICAL)

# Define el prompt con el formato adecuado
prompt = """Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. Responde únicamente con la emoción correspondiente.

Texto: "Me indigna ver cómo algunas personas menosprecian los derechos humanos"
"""

# Genera el resultado usando el pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")

# Muestra el resultado
print(result[0]['generated_text'])

<s>[INST] Clasifica el siguiente texto en una de estas emociones: ira, disgusto, tristeza, alegría, miedo o neutral. Responde únicamente con la emoción correspondiente.

Texto: "Me indigna ver cómo algunas personas menosprecian los derechos humanos"
 [/INST] ira 


In [29]:
# Reload model in FP16 and merge it with LoRA weights
import gc
from peft import PeftModel  # Import PeftModel
!pip install safetensors
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

gc.collect()
torch.cuda.empty_cache()  # Clear the GPU cache

# Specify offload folder
offload_folder = "./offload"  # Or any directory you prefer
os.makedirs(offload_folder, exist_ok=True)  # Create if it doesn't exist

# Configure BitsAndBytes for 8-bit loading with CPU offload
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offload for 32-bit parts
)

# Load the model using the BitsAndBytes config
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",  # Load on CPU or use device_map="auto"
    offload_folder=offload_folder,  # Specify offload folder
    quantization_config=bnb_config,  # Use the BitsAndBytes config
)

# Pass the offload_folder to PeftModel.from_pretrained
model = PeftModel.from_pretrained(load_model, '/kaggle/working/llama-2-7b-emotion-detector-v5', offload_folder=offload_folder)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Delete the load_model to free up memory
del load_model
gc.collect()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

90

In [32]:
# from huggingface_hub import create_repo, upload_folder

# # Ruta local de tu checkpoint LoRA
# lora_weights_path = "/kaggle/working/Llama-2-7b-ft-instruct-es-fine-tuned-testing5/checkpoint-6732------------"
# # El repo que quieres crear en HF
# repo_id = "Joseph7D/llama-2-7b-emotion-detector-v5"

# # Crea el repo (si ya existe, exist_ok=True lo ignora)
# create_repo(repo_id, repo_type="model", exist_ok=True)

# # Sube TODO el contenido de tu carpeta a la raíz del repo
# upload_folder(
#     folder_path = lora_weights_path,
#     path_in_repo = "",       # lo sube directamente al root del repo
#     repo_id     = repo_id,
#     repo_type   = "model",
#     commit_message="🚀 Subiendo adaptador LoRA"
# )

# print(f"✅ Archivos LoRA subidos a https://huggingface.co/{repo_id}")


rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

scaler.pt:   0%|          | 0.00/988 [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/320M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

✅ Archivos LoRA subidos a https://huggingface.co/Joseph7D/llama-2-7b-emotion-detector-v3


In [34]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(
#     "clibrain/Llama-2-7b-ft-instruct-es",
#     trust_remote_code=True
# )

# # Esto crea un directorio temporal, vuelca ahí el tokenizer, y lo sube.
# tokenizer.push_to_hub(
#     repo_id, 
#     use_temp_dir=True, 
#     commit_message="Añadiendo tokenizer"
# )

# print("✅ Tokenizer subido.")


No files have been modified since last commit. Skipping to prevent empty commit.


✅ Tokenizer subido.


In [30]:

# Update generation config
model.generation_config.temperature = None  # Unset temperature
model.generation_config.top_p = None  # Unset top_p

# Reload tokenizer to save it
# ... (rest of the code remains the same) ...

model.push_to_hub(new_model, use_temp_dir=True)
tokenizer.push_to_hub(new_model, use_temp_dir=True)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Joseph7D/llama-2-7b-emotion-detector-v5/commit/b4bd57022d048c54836371b33e581e9015131203', commit_message='Upload tokenizer', commit_description='', oid='b4bd57022d048c54836371b33e581e9015131203', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Joseph7D/llama-2-7b-emotion-detector-v5', endpoint='https://huggingface.co', repo_type='model', repo_id='Joseph7D/llama-2-7b-emotion-detector-v5'), pr_revision=None, pr_num=None)